In [31]:
# neccessary libraries
import pymysql
import pandas as pd

In [32]:
# connection to database 
db_name = "fraudanalysis"
db_host = "127.0.0.1"
db_username = "root"
db_password = "myself"

try:
    conn = pymysql.connect(host = db_host,
                           port = int(3306),
                           user = "root",
                           password = db_password,
                           db = db_name)
except e:
    print (e)
if conn:
    print ("connection successful")
else:
        print ("error")

connection successful


# Finding Outliers

In [33]:
def execute_query(query):

    transaction_df = pd.read_sql(sql=query, con=conn)

    return transaction_df

In [34]:
# join whole database into a dataframe

query = """
SELECT 
    a.CustomerID,
    a.AccountBalance, 
    a.LastLogin, 
    
    b.TransactionID,
    b.TransactionAmount, 
    
    c.AnomalyScore, 
    
    d.Address,
    d.Age,
    d.Name,
    
    e.FraudIndicator, 
    
    f.Location,
    f.MerchantID,
    
    g.SuspiciousFlag,
    
    h.Category,
    
    i.Timestamp,
    
    j.Amount
FROM 
    fraudanalysis.account_activity a
JOIN 
    fraudanalysis.transaction_records j ON a.CustomerID = j.CustomerID
JOIN 
    fraudanalysis.amount_data b ON j.TransactionID = b.TransactionID
LEFT JOIN 
    fraudanalysis.anomaly_scores c ON b.TransactionID = c.TransactionID
JOIN 
    fraudanalysis.customer_data d ON a.CustomerID = d.CustomerID
LEFT JOIN 
    fraudanalysis.fraud_indicators e ON b.TransactionID = e.TransactionID
JOIN 
    fraudanalysis.transaction_metadata i ON b.TransactionID = i.TransactionID
JOIN 
    fraudanalysis.merchant_data f ON i.MerchantID = f.MerchantID
LEFT JOIN 
    fraudanalysis.suspicious_activity g ON a.CustomerID = g.CustomerID
LEFT JOIN 
    fraudanalysis.transaction_category_labels h ON b.TransactionID = h.TransactionID;
"""


In [21]:
transaction_df = execute_query(query)

C:\Users\user\Documents\PythonInstaller\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
transaction_df

,CustomerID,AccountBalance,LastLogin,TransactionID,TransactionAmount,AnomalyScore,Address,Age,Name,FraudIndicator,Location,MerchantID,SuspiciousFlag,Category,Timestamp,Amount
0,1924,2652.931615,2024-07-12,816,54.402165,0.209246,Address 1924,60,Customer 1924,0,Location 2001,2001,0,Online,2022-02-03 23:00:00,26.521145
1,1730,3967.076794,2023-12-31,623,99.095151,0.840506,Address 1730,54,Customer 1730,0,Location 2004,2004,0,Food,2022-01-26 22:00:00,75.262308
2,1167,4121.316625,2022-06-16,430,77.485946,0.096968,Address 1167,33,Customer 1167,0,Location 2004,2004,0,Retail,2022-01-18 21:00:00,26.733576
3,1995,2240.517799,2024-09-21,262,18.327174,0.387238,Address 1995,38,Customer 1995,0,Location 2005,2005,0,Retail,2022-01-11 21:00:00,54.474330
4,1228,9343.248065,2022-08-16,910,93.896547,0.661051,Address 1228,59,Customer 1228,0,Location 2007,2007,1,Food,2022-02-07 21:00:00,32.231877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1038,5402.920850,2022-02-07,40,89.313300,0.532609,Address 1038,62,Customer 1038,0,Location 2995,2995,0,Other,2022-01-02 15:00:00,80.537776
996,1091,1736.468439,2022-04-01,772,94.503233,0.668801,Address 1091,58,Customer 1091,0,Location 2997,2997,0,Online,2022-02-02 03:00:00,56.424874
997,1304,9631.517578,2022-10-31,420,38.960782,0.006909,Address 1304,62,Customer 1304,0,Location 2997,2997,0,Retail,2022-01-18 11:00:00,79.336276
998,1127,4843.290416,2022-05-07,165,39.624162,0.063965,Address 1127,39,Customer 1127,0,Location 2998,2998,0,Food,2022-01-07 20:00:00,70.658074


In [23]:
transaction_df[~transaction_df.isnull().any(axis=1)]

,CustomerID,AccountBalance,LastLogin,TransactionID,TransactionAmount,AnomalyScore,Address,Age,Name,FraudIndicator,Location,MerchantID,SuspiciousFlag,Category,Timestamp,Amount
0,1924,2652.931615,2024-07-12,816,54.402165,0.209246,Address 1924,60,Customer 1924,0,Location 2001,2001,0,Online,2022-02-03 23:00:00,26.521145
1,1730,3967.076794,2023-12-31,623,99.095151,0.840506,Address 1730,54,Customer 1730,0,Location 2004,2004,0,Food,2022-01-26 22:00:00,75.262308
2,1167,4121.316625,2022-06-16,430,77.485946,0.096968,Address 1167,33,Customer 1167,0,Location 2004,2004,0,Retail,2022-01-18 21:00:00,26.733576
3,1995,2240.517799,2024-09-21,262,18.327174,0.387238,Address 1995,38,Customer 1995,0,Location 2005,2005,0,Retail,2022-01-11 21:00:00,54.474330
4,1228,9343.248065,2022-08-16,910,93.896547,0.661051,Address 1228,59,Customer 1228,0,Location 2007,2007,1,Food,2022-02-07 21:00:00,32.231877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1038,5402.920850,2022-02-07,40,89.313300,0.532609,Address 1038,62,Customer 1038,0,Location 2995,2995,0,Other,2022-01-02 15:00:00,80.537776
996,1091,1736.468439,2022-04-01,772,94.503233,0.668801,Address 1091,58,Customer 1091,0,Location 2997,2997,0,Online,2022-02-02 03:00:00,56.424874
997,1304,9631.517578,2022-10-31,420,38.960782,0.006909,Address 1304,62,Customer 1304,0,Location 2997,2997,0,Retail,2022-01-18 11:00:00,79.336276
998,1127,4843.290416,2022-05-07,165,39.624162,0.063965,Address 1127,39,Customer 1127,0,Location 2998,2998,0,Food,2022-01-07 20:00:00,70.658074


## Check for NaN Values

In [35]:
# check for NA values
for column in transaction_df.columns:
    if transaction_df[column].isna().any():
        print(f"Column '{column}' has NA values.")
    else:
        pass

## Check for outliers

In [36]:
# subset dataframe with relevant columns

financial_df = transaction_df[["AccountBalance", "TransactionAmount", "AnomalyScore", "Age", "Amount"]]

In [37]:
financial_df

,AccountBalance,TransactionAmount,AnomalyScore,Age,Amount
0,2652.931615,54.402165,0.209246,60,26.521145
1,3967.076794,99.095151,0.840506,54,75.262308
2,4121.316625,77.485946,0.096968,33,26.733576
3,2240.517799,18.327174,0.387238,38,54.474330
4,9343.248065,93.896547,0.661051,59,32.231877
...,...,...,...,...,...
995,5402.920850,89.313300,0.532609,62,80.537776
996,1736.468439,94.503233,0.668801,58,56.424874
997,9631.517578,38.960782,0.006909,62,79.336276
998,4843.290416,39.624162,0.063965,39,70.658074


In [41]:
for column in financial_df:
    Q1 = financial_df[column].quantile(0.25)
    Q3 = financial_df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    threshold = 1.5
    outliers = financial_df[(financial_df[column] < Q1 - threshold * IQR) | (financial_df[column] > Q3 + threshold * IQR)]
    
    if not outliers.empty:
        print(f"Outliers in column '{column}':")
        print(outliers)
    else:
        print(f"No outliers found in column '{column}'.")
    

No outliers found in column 'AccountBalance'.
No outliers found in column 'TransactionAmount'.
No outliers found in column 'AnomalyScore'.
No outliers found in column 'Age'.
No outliers found in column 'Amount'.
